In [109]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 



In [110]:
df = pd.read_csv ( "D:\Python basics\PHARMAai\indian_pharmaceutical_products_clean.csv")

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Saurav\AppData\Local\Temp\ipykernel_5684\1617714729.py:1: SyntaxWarning: invalid escape sequence '\P'
  df = pd.read_csv ( "D:\Python basics\PHARMAai\indian_pharmaceutical_products_clean.csv")


In [111]:
df.isnull().sum()

product_id                    0
brand_name                    0
manufacturer                  0
price_inr                     0
is_discontinued               0
dosage_form                   0
pack_size                 22330
pack_unit                 22330
num_active_ingredients        0
primary_ingredient            0
primary_strength          25198
active_ingredients            0
therapeutic_class             0
packaging_raw                 0
manufacturer_raw              0
dtype: int64

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253973 entries, 0 to 253972
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   product_id              253973 non-null  int64  
 1   brand_name              253973 non-null  object 
 2   manufacturer            253973 non-null  object 
 3   price_inr               253973 non-null  float64
 4   is_discontinued         253973 non-null  bool   
 5   dosage_form             253973 non-null  object 
 6   pack_size               231643 non-null  float64
 7   pack_unit               231643 non-null  object 
 8   num_active_ingredients  253973 non-null  int64  
 9   primary_ingredient      253973 non-null  object 
 10  primary_strength        228775 non-null  object 
 11  active_ingredients      253973 non-null  object 
 12  therapeutic_class       253973 non-null  object 
 13  packaging_raw           253973 non-null  object 
 14  manufacturer_raw    

In [113]:
df.columns

Index(['product_id', 'brand_name', 'manufacturer', 'price_inr',
       'is_discontinued', 'dosage_form', 'pack_size', 'pack_unit',
       'num_active_ingredients', 'primary_ingredient', 'primary_strength',
       'active_ingredients', 'therapeutic_class', 'packaging_raw',
       'manufacturer_raw'],
      dtype='object')

In [114]:
df['pack_unit'].value_counts()

pack_unit
strip     172122
bottle     33772
ml         14790
tube        8241
units       1424
gm          1294
Name: count, dtype: int64

In [115]:
import pandas as pd
import re



# Check initial missing values in 'pack_unit'
initial_missing = df['pack_unit'].isnull().sum()
print(f"Initial missing values in 'pack_unit': {initial_missing}")

# Define a function to extract pack unit from 'packaging_raw'
def extract_pack_unit(row):
    # If pack_unit is already present, return it
    if pd.notna(row['pack_unit']):
        return row['pack_unit']
    
    # Check for keywords in a case-insensitive manner
    packaging_text = str(row['packaging_raw']).lower()
    
    if re.search(r'bottle', packaging_text):
        return 'bottle'
    elif re.search(r'strip|blister', packaging_text):
        # We combine 'strip' and 'blister' as they are similar
        return 'strip'
    elif re.search(r'box|carton', packaging_text):
        # We combine 'box' and 'carton'
        return 'box'
    elif re.search(r'pouch', packaging_text):
        return 'pouch'
    elif re.search(r'vial', packaging_text):
        return 'vial'
    elif re.search(r'tube', packaging_text):
        return 'tube'
    elif re.search(r'sachet', packaging_text):
        return 'sachet'
    elif re.search(r'ampule|ampoule', packaging_text):
        return 'ampoule'
    else:
        # Return a missing value if no match is found
        return None

# Apply the function to the DataFrame
df['pack_unit'] = df.apply(extract_pack_unit, axis=1)

# Check for missing values after imputation
final_missing = df['pack_unit'].isnull().sum()
print(f"Missing values after imputation: {final_missing}")




Initial missing values in 'pack_unit': 22330
Missing values after imputation: 997


In [116]:
initial_missing = df['pack_size'].isnull().sum()
print(f"Initial missing values in 'pack_size': {initial_missing}")

# Define a function to extract pack size from 'packaging_raw'
def extract_pack_size(row):
    # If pack_size is already present, return it
    if pd.notna(row['pack_size']):
        return row['pack_size']
    
    # Heuristic: If dosage form is 'injection', assume pack size is 1.
    if str(row['dosage_form']).lower() == 'injection':
        return 1.0
        
    # Check for a number followed by a keyword in a case-insensitive manner
    packaging_text = str(row['packaging_raw']).lower()
    
    # Common patterns: '10 tablets', '500 mg', '100 ml', 'strip of 10'
    match = re.search(r'(\d+)\s*(tablet|cap|unit|ml|gm|g|pouch|sachet|vial|strip|pills|bottle)', packaging_text)
    
    if match:
        # Return the captured number as a float
        return float(match.group(1))
    else:
        # Return a missing value if no match is found
        return None

# Apply the function to the DataFrame
df['pack_size'] = df.apply(extract_pack_size, axis=1)

# Check for missing values after imputation
final_missing = df['pack_size'].isnull().sum()
print(f"Missing values after imputation: {final_missing}")


Initial missing values in 'pack_size': 22330
Missing values after imputation: 2480


In [117]:
import pandas as pd
import ast


# Check initial missing values in 'primary_strength'
initial_missing = df['primary_strength'].isnull().sum()
print(f"Initial missing values in 'primary_strength': {initial_missing}")

# Define a function to extract primary strength from the active_ingredients list
def extract_from_ingredients(row):
    # If primary_strength is already present, return it
    if pd.notna(row['primary_strength']):
        return row['primary_strength']
    
    # Try to safely evaluate the string representation of the list
    try:
        ingredients_list = ast.literal_eval(row['active_ingredients'])
        primary_ingredient = str(row['primary_ingredient']).lower()
        
        for ingredient in ingredients_list:
            if str(ingredient['name']).lower() == primary_ingredient:
                return ingredient['strength']
    except (ValueError, SyntaxError, TypeError):
        pass
    
    return None

# Apply the extraction function to the DataFrame
df['primary_strength'] = df.apply(extract_from_ingredients, axis=1)

# Impute remaining missing values with the mode of their group
# First, calculate the mode for each group
grouped_modes = df.groupby(['primary_ingredient', 'dosage_form'])['primary_strength'].transform(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
)

# Then, use the calculated modes to fill the NaN values
df['primary_strength'].fillna(grouped_modes, inplace=True)

# Check for missing values after imputation
final_missing = df['primary_strength'].isnull().sum()
print(f"Missing values after imputation: {final_missing}")




Initial missing values in 'primary_strength': 25198
Missing values after imputation: 5114


C:\Users\Saurav\AppData\Local\Temp\ipykernel_5684\2906072008.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['primary_strength'].fillna(grouped_modes, inplace=True)


In [118]:
df

,product_id,brand_name,manufacturer,price_inr,is_discontinued,dosage_form,pack_size,pack_unit,num_active_ingredients,primary_ingredient,primary_strength,active_ingredients,therapeutic_class,packaging_raw,manufacturer_raw
0,1,Augmentin 625 Duo Tablet,Glaxo SmithKline Pharmaceuticals Ltd,223.42,False,tablet,10.0,strip,2,Amoxycillin,500mg,"[{'name': 'Amoxycillin', 'strength': '500mg', ...",antibiotic,strip of 10 tablets,Glaxo SmithKline Pharmaceuticals Ltd
1,2,Azithral 500 Tablet,Alembic Pharmaceuticals Ltd,132.36,False,tablet,5.0,strip,1,Azithromycin,500mg,"[{'name': 'Azithromycin', 'strength': '500mg',...",antibiotic,strip of 5 tablets,Alembic Pharmaceuticals Ltd
2,3,Ascoril LS Syrup,Glenmark Pharmaceuticals Ltd,118.00,False,syrup,100.0,bottle,2,Ambroxol,30mg/5ml,"[{'name': 'Ambroxol', 'strength': '30mg/5ml', ...",bronchodilator,bottle of 100 ml Syrup,Glenmark Pharmaceuticals Ltd
3,4,Allegra 120mg Tablet,Sanofi India Ltd,218.81,False,tablet,10.0,strip,1,Fexofenadine,120mg,"[{'name': 'Fexofenadine', 'strength': '120mg',...",antihistamine,strip of 10 tablets,Sanofi India Ltd
4,5,Avil 25 Tablet,Sanofi India Ltd,10.96,False,tablet,15.0,strip,1,Pheniramine,25mg,"[{'name': 'Pheniramine', 'strength': '25mg', '...",other,strip of 15 tablets,Sanofi India Ltd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253968,253969,Ziyapod 100mg Oral Suspension,Ziyana Lifesciences Pvt Ltd,62.30,False,suspension,30.0,bottle,1,Cefpodoxime Proxetil,100mg,"[{'name': 'Cefpodoxime Proxetil', 'strength': ...",other,bottle of 30 ml Oral Suspension,Ziyana Lifesciences Pvt Ltd
253969,253970,Zemhart 30mg Tablet,Leeford Healthcare Ltd,54.00,False,tablet,10.0,strip,1,Diltiazem,30mg,"[{'name': 'Diltiazem', 'strength': '30mg', 'fu...",other,strip of 10 tablets,Leeford Healthcare Ltd
253970,253971,Zivex 25mg Tablet,Euro Organics,57.00,False,tablet,10.0,strip,1,Hydroxyzine,25mg,"[{'name': 'Hydroxyzine', 'strength': '25mg', '...",other,strip of 10 tablets,Euro Organics
253971,253972,ZI Fast 500mg Injection,Burgeon Health Series Private Limited,152.00,False,injection,1.0,vial,1,Azithromycin,500mg,"[{'name': 'Azithromycin', 'strength': '500mg',...",antibiotic,vial of 1 Injection,Burgeon Health Series Private Limited


In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253973 entries, 0 to 253972
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   product_id              253973 non-null  int64  
 1   brand_name              253973 non-null  object 
 2   manufacturer            253973 non-null  object 
 3   price_inr               253973 non-null  float64
 4   is_discontinued         253973 non-null  bool   
 5   dosage_form             253973 non-null  object 
 6   pack_size               251493 non-null  float64
 7   pack_unit               252976 non-null  object 
 8   num_active_ingredients  253973 non-null  int64  
 9   primary_ingredient      253973 non-null  object 
 10  primary_strength        248859 non-null  object 
 11  active_ingredients      253973 non-null  object 
 12  therapeutic_class       253973 non-null  object 
 13  packaging_raw           253973 non-null  object 
 14  manufacturer_raw    

In [120]:
df.isnull().sum()

product_id                   0
brand_name                   0
manufacturer                 0
price_inr                    0
is_discontinued              0
dosage_form                  0
pack_size                 2480
pack_unit                  997
num_active_ingredients       0
primary_ingredient           0
primary_strength          5114
active_ingredients           0
therapeutic_class            0
packaging_raw                0
manufacturer_raw             0
dtype: int64